Загрузим всё необходимое

In [3]:
import gzip
import numpy as np
import time
import idx2numpy

Подготовим данные тренировочного набора MNIST для работы:

In [4]:
class_count = 10

x_train = idx2numpy.convert_from_file('train-images.idx3-ubyte')
y_train = idx2numpy.convert_from_file('train-labels.idx1-ubyte')
x_test = idx2numpy.convert_from_file('t10k-images.idx3-ubyte')
y_test = idx2numpy.convert_from_file('t10k-labels.idx1-ubyte')

x_train = x_train / 255.0
x_test = x_test / 255.0

x_train = x_train.reshape((x_train.shape[0], 28 * 28))
x_test = x_test.reshape((x_test.shape[0], 28 * 28))

y_train = np.eye(class_count)[y_train]
y_test = np.eye(class_count)[y_test]

print("Размеры тренировочных данных для x и y:", x_train.shape, "и", y_train.shape, "соответственно")
print("Размеры тестовых данных для x и y:", x_test.shape, "и", x_test.shape, "соответственно")

Размеры тренировочных данных для x и y: (60000, 784) и (60000, 10) соответственно
Размеры тестовых данных для x и y: (10000, 784) и (10000, 784) соответственно


Определим класс NeuralNetwork, в котором будет реализован метод обратного распространения ошибки:

In [5]:
class NeuralNetwork(object):
    def __init__(self, input_layer, hidden_layer, output_layer):
        super(NeuralNetwork, self).__init__()
        # Вход
        self.input_layer = input_layer
        # Скрытый слой
        self.hidden_layer = hidden_layer
        # Выход
        self.output_layer = output_layer

        # Синаптические веса (входные сигналы от других нейронов)
        self.w = [np.random.normal(0, np.sqrt(2 / (input_layer)), (input_layer, hidden_layer)),
                  np.random.normal(0, np.sqrt(6 / (hidden_layer + output_layer)), (hidden_layer, output_layer))]

        # Воздействие внешней среды (сдвиги), представляется свободным членом
        self.b = [np.zeros(hidden_layer),
                  np.zeros(output_layer)]

    # Функция активации Relu (положительная срезка) для внутренного слоя
    def relu(self, x):
        return np.maximum(x, 0)

    # Производная функции Relu
    def derived_relu(self, x):
        return np.where(x > 0.0, 1, 0)

    # Функция активации SoftMax для внешного слоя
    def softmax(self, x):
        exp_x = np.exp(x)
        return np.divide(exp_x.T, (np.sum(exp_x, axis = 1))).T

    # Точность
    def compute_accuracy(self, y_true, y_pred):
        return np.mean(np.argmax(y_true, axis=1) == np.argmax(y_pred, axis=1))

    # Функция ошибки - кросс-энтропия для задачи классификации
    def compute_cross_entropy_loss(self, y_true, y_pred):
        return np.mean(-np.sum(y_true * np.log(y_pred), axis=1))

    # Прямой проход
    def forward(self, x):
        # Сумма синапсов с некоторыми весами и внешнего воздействия
        x = np.matmul(x, self.w[0]) + self.b[0]
        # Запоминаем значения, полученные на внутреннем слое
        self.inner_layer_w = x.copy()

        # Преобразования сигнала с помощью функции активации (цель - ограничить амплитуду выходного сигнала)
        x = self.relu(x)
        # Запоминаем веса, полученные после активации
        self.h = x.copy()

        # То же самое, но от скрытого слоя к выходному
        x = np.matmul(x, self.w[1]) + self.b[1]
        # Функции активации
        x = self.softmax(x)
        return x

    # Обратный проход
    # На базе минипакетного стохастического градиентного спуска
    def backward(self, x_train, y_pred, y_true): # Обратный проход
        
        # Делим единицу на число примеров в пакете (не забываем про минус)
        one_devide_l = - 1 / x_train.shape[0]
        
        d_z2 = y_pred - y_true

        # Частная производная по весам w2 связей от скрытого слоя к выходному
        # Производная логарифма сокращается с производной Softmax, которая равна сама себе
        # Из производной сложной функции по итогу остаётся только Relu
        d_w2 = one_devide_l * np.matmul(self.h.T, d_z2)

        # Частная производная по весам w1 связей от входного слоя к скрытому
        # Производная логарифма сокращается с производной Softmax, которая равна сама себе
        # Веса w2 выносятся как константа перед производной
        # Производная от сложной Relu даёт производной себя, помноженную на x
        d_z1 = np.matmul(d_z2, self.w[1].T) * self.derived_relu(self.inner_layer_w)
        d_w1 = one_devide_l * np.matmul(x_train.T, d_z1)

        # Частная производная по сдвигам b2 и b1.
        # Почти повторяет dw2 и dw1, но производные сложных функций дают просто Softmax * 1 и Relu' * 1
        d_b2 = one_devide_l * np.sum(d_z2, axis=0)
        d_b1 = one_devide_l * np.sum(d_z1, axis=0)

        # Корректировка синаптических весов
        self.w[1] = self.w[1] + self.learning_rate * d_w2
        self.w[0] = self.w[0] + self.learning_rate * d_w1
        self.b[1] = self.b[1] + self.learning_rate * d_b2
        self.b[0] = self.b[0] + self.learning_rate * d_b1

    def train(self, x_train, y_train, x_test, y_test, epochs, learning_rate, batch_size):
        self.learning_rate = learning_rate

        start_of_all_train_time = time.time()
        for epoch in range(epochs):

            start_of_train_time = time.time()

            step_count = x_train.shape[0] // batch_size
            if x_train.shape[0] % batch_size > 0:
                step_count += 1

            for batch_id in range(step_count):
                start_id = batch_id * batch_size
                finish_id = min((batch_id + 1) * batch_size, y_train.shape[0])

                y_pred = self.forward(x_train[start_id:finish_id])
                self.backward(x_train[start_id:finish_id], y_pred, y_train[start_id:finish_id])

            end_of_train_time = time.time()
            epoch_train_time = end_of_train_time - start_of_train_time

            y_pred = self.forward(x_train)
            error_train = self.compute_cross_entropy_loss(y_train, y_pred)
            acc_train = self.compute_accuracy(y_train, y_pred)
            
            print("Эпоха №", (epoch + 1), end='')
            print(f"\tВремя обучение: {epoch_train_time:.2f} сек", end='')
            print(f"\tОшибка: {error_train:.2f}", end='')
            print(f"\tТочность: {acc_train:.2f}")

        end_of_all_train_time = time.time()
        all_train_time = end_of_all_train_time - start_of_all_train_time
        print(f"Общее время обучение: {all_train_time:.2f} сек\n")
            
        y_pred = self.forward(x_test)
        error_test = self.compute_cross_entropy_loss(y_test, y_pred)
        acc_test = self.compute_accuracy(y_test, y_pred)

        print(f"Ошибка на тестовой выборке: {error_test:.2f}")
        print(f"Точность на тестовой выборке: {acc_test:.2f}")

Обучим модель:

In [6]:
output_layer = class_count

batch_size = 64
hidden_layer = 300
lr = 0.1
epoch_count = 20

nn = NeuralNetwork(x_train.shape[1], hidden_layer, output_layer)
nn.train(x_train, y_train, x_test, y_test, epoch_count, lr, batch_size)

Эпоха № 1	Время обучение: 4.18 сек	Ошибка: 0.20	Точность: 0.94
Эпоха № 2	Время обучение: 3.75 сек	Ошибка: 0.14	Точность: 0.96
Эпоха № 3	Время обучение: 3.56 сек	Ошибка: 0.11	Точность: 0.97
Эпоха № 4	Время обучение: 3.58 сек	Ошибка: 0.09	Точность: 0.98
Эпоха № 5	Время обучение: 3.54 сек	Ошибка: 0.07	Точность: 0.98
Эпоха № 6	Время обучение: 4.07 сек	Ошибка: 0.06	Точность: 0.98
Эпоха № 7	Время обучение: 3.97 сек	Ошибка: 0.06	Точность: 0.99
Эпоха № 8	Время обучение: 4.45 сек	Ошибка: 0.05	Точность: 0.99
Эпоха № 9	Время обучение: 3.55 сек	Ошибка: 0.04	Точность: 0.99
Эпоха № 10	Время обучение: 3.54 сек	Ошибка: 0.04	Точность: 0.99
Эпоха № 11	Время обучение: 3.54 сек	Ошибка: 0.04	Точность: 0.99
Эпоха № 12	Время обучение: 3.68 сек	Ошибка: 0.03	Точность: 0.99
Эпоха № 13	Время обучение: 3.50 сек	Ошибка: 0.03	Точность: 0.99
Эпоха № 14	Время обучение: 3.36 сек	Ошибка: 0.03	Точность: 0.99
Эпоха № 15	Время обучение: 3.46 сек	Ошибка: 0.03	Точность: 0.99
Эпоха № 16	Время обучение: 3.35 сек	Ошибка: 0.02	